In [2]:
import pandas as pd

# Read in the data
df = pd.read_json("data/2017.json")
df.head()

,Grade,UserRating,Moves
0,6A+,1,"[H5, E7, F8, D10, E13, C14, C16, B18, E18, F9,..."
1,6A+,2,"[H5, E7, D8, D10, E13, C14, C16, B18, E18]"
2,6B+,2,"[A4, B4, C7, D9, F12, D15, F18]"
3,6A+,2,"[K4, J4, I7, H9, I10, F12, F13, E6, B14, C16, ..."
4,6B+,2,"[G8, D9, F12, C13, F15, D17, F18, E6, G4, F4]"


In [8]:
# Moves is a column with an array of moves, each indicated as the hold coordinates A1, B2 etc.

# Create a hisogram of each hold coordinate and the Grades associated with it 6A, 6A+, 6B etc.

hold_x = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]
hold_y = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

coorindates = []
for x in hold_x:
    for y in hold_y:
        coorindates.append(x + str(y))

# convert the array to a dataframe
df_coordinates = pd.DataFrame(coorindates, columns=["coordinates"])
df_coordinates.head()

# Grades
grades = ["6A", "6A+", "6B", "6B+", "6C", "6C+", "7A", "7A+", "7B", "7B+", "7C", "7C+", "8A", "8A+", "8B", "8B+", "8C", "8C+"]

# add grades to the dataframe as columns
for grade in grades:
    df_coordinates[grade] = 0

# for each move in df['Moves'] add 1 to the grade column which corresponds in the df['Grade']
for index, row in df.iterrows():
    for move in row['Moves']:
        df_coordinates.loc[df_coordinates['coordinates'] == move, row['Grade']] += 1



In [10]:
df_coordinates

,coordinates,6A,6A+,6B,6B+,6C,6C+,7A,7A+,7B,7B+,7C,7C+,8A,8A+,8B,8B+,8C,8C+
0,A1,0,18,4,10,3,10,6,9,1,5,2,2,1,0,0,0,0,0
1,A2,0,12,3,11,12,7,15,8,5,3,2,0,0,1,0,0,0,0
2,A3,0,62,44,57,31,62,53,30,8,19,14,8,3,0,0,1,0,0
3,A4,0,64,49,83,42,52,63,50,17,25,15,5,2,0,0,0,0,0
4,A5,0,384,154,230,90,188,131,80,27,41,27,9,2,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,K14,0,12,5,15,10,12,27,16,10,12,8,3,4,0,0,0,0,0
194,K15,0,167,51,84,25,58,46,24,6,14,8,3,0,0,0,0,0,0
195,K16,0,10,7,11,9,18,18,6,3,3,4,3,1,0,0,0,0,0
196,K17,0,24,14,18,12,16,6,8,4,2,4,1,0,0,0,0,0,0


In [19]:
# assign a hold difficulty rating based upon the grades associated with it
multiplier = {"6A": 1, "6A+": 2, "6B": 3, "6B+": 4, "6C": 5, "6C+": 6, "7A": 7, "7A+": 8, "7B": 9, "7B+": 10, "7C": 11, "7C+": 12, "8A": 13, "8A+": 14, "8B": 15, "8B+": 16, "8C": 17, "8C+": 18}
# exponential scale multiplier
exp_multiplier = {"6A": 1, "6A+": 2, "6B": 4, "6B+": 8, "6C": 16, "6C+": 32, "7A": 64, "7A+": 128, "7B": 256, "7B+": 512, "7C": 1024, "7C+": 2048, "8A": 4096, "8A+": 8192, "8B": 16384, "8B+": 32768, "8C": 65536, "8C+": 131072}

# add a column to the dataframe with the hold difficulty rating
df_coordinates['hold_pop_diff'] = 0

# for each grade column, multiply the grade by the multiplier and add it to the hold_pop_diff column
for i, grade in enumerate(grades):
    if(i > 7):
      df_coordinates['hold_pop_diff'] += df_coordinates[grade] * multiplier[grade]
    else:
      df_coordinates['hold_pop_diff'] -= df_coordinates[grade] * multiplier[grade]
    

# Sort the dataframe by hold_pop_diff
df_coordinates.sort_values(by=['hold_pop_diff'], ascending=False, inplace=True)

# normalise to be all positive values between 0 and 1
df_coordinates['hold_pop_diff'] = (df_coordinates['hold_pop_diff'] - df_coordinates['hold_pop_diff'].min()) / (df_coordinates['hold_pop_diff'].max() - df_coordinates['hold_pop_diff'].min())

df_coordinates.head(20)

,coordinates,6A,6A+,6B,6B+,6C,6C+,7A,7A+,7B,...,7C,7C+,8A,8A+,8B,8B+,8C,8C+,hold_difficulty,hold_pop_diff
160,I17,0,7,0,6,3,7,18,14,13,...,13,5,3,0,0,1,0,0,426,1.000000
191,K12,0,9,3,7,6,11,20,18,9,...,21,6,1,0,0,1,0,0,456,0.997442
173,J12,0,4,6,10,4,17,30,27,9,...,16,7,3,2,1,0,0,0,501,0.989995
12,A13,0,12,1,6,6,12,17,14,7,...,18,3,2,1,1,0,0,0,282,0.989168
18,B1,0,4,2,4,1,4,4,4,4,...,1,1,1,0,0,0,0,0,67,0.985031
34,B17,0,9,4,3,2,6,8,11,3,...,5,4,4,0,0,0,0,0,156,0.983301
33,B16,0,13,4,5,6,21,25,14,15,...,12,5,1,1,1,0,0,0,202,0.983150
36,C1,0,4,2,9,5,10,8,4,1,...,2,2,1,0,0,0,0,0,29,0.982172
67,D14,0,24,4,12,5,12,31,19,12,...,15,7,0,0,1,0,0,0,252,0.980894
6,A7,0,9,6,11,5,6,10,9,4,...,7,0,0,1,0,0,0,0,88,0.980593


In [21]:
hold_dist_df = pd.read_csv("exports/hold_dist_df_40.csv")
hold_dist_df.head()

,number,A0,A1,A2,A3,A4,A5,A6,A7,A8,...,K9,K10,K11,K12,K13,K14,K15,K16,K17,K18
0,A0,0,175,350,525,700,875,1050,1225,1400,...,2546,2657,2776,2900,3029,3162,3300,3441,3584,3731
1,A1,175,0,175,350,525,700,875,1050,1225,...,2441,2546,2657,2776,2900,3029,3162,3300,3441,3584
2,A2,350,175,0,175,350,525,700,875,1050,...,2345,2441,2546,2657,2776,2900,3029,3162,3300,3441
3,A3,525,350,175,0,175,350,525,700,875,...,2259,2345,2441,2546,2657,2776,2900,3029,3162,3300
4,A4,700,525,350,175,0,175,350,525,700,...,2183,2259,2345,2441,2546,2657,2776,2900,3029,3162


In [40]:
# sort df['Moves'] by ascending order in terms of inter so the largest numbers are at the end, and the smallest at the start
# first split A1 into i.e - [A, 1] then sort by the first element, then by the second element, so that A1 is before A5 and K1 is before A4 and A18 and K18 end up near the end or last
df['Moves'] = df['Moves'].apply(lambda x: sorted(x, key=lambda y: (int(y[1:]), y[0])))

df.head()

,Grade,UserRating,Moves,total_move_dist,largest_move,hardest_move
13935,6C+,3,"[A1, A6, B8, E8, C13, G15, E16, J16, H18, K18]",10852.0,3584.0,3396.631262
2883,7A+,0,"[A1, D1, C4, K4, B8, K9, G11, H11, J16, K18]",10846.0,3584.0,3396.631262
3494,8A,3,"[A1, K1, F10, A18, K18]",10898.0,3584.0,3396.631262
18737,6B+,2,"[K1, J6, F8, I8, F12, C13, C16, A18]",9203.0,3584.0,3306.586430
4105,6C+,3,"[A1, A5, C10, A14, E16, E17, H18]",7562.0,3288.0,3198.714006


In [41]:
# Assuming the df['Moves'] is a list of moves in order, we can use this to calculate the distance between each move
# and the previous move. This will give us a list of distances between each move.
# We should also look for the largest single move as a difficulty guage

# in hold_dist_df the first column contains the coorindates (i.e - A1, A2 etc) and the remaining columns are also the coordinates
for index, row in df.iterrows():
    # add a new column to the dataframe with the total of distances between each move
    df.loc[index, 'total_move_dist'] = 0
    # add a new column to the dataframe with the largest single move
    df.loc[index, 'largest_move'] = 0
    # 
    df.loc[index, 'hardest_move'] = 0
    for move_index, move in enumerate(row['Moves']):
        if(move_index > 0):
            # calculate the distance between the previous move and the current move
            dist = hold_dist_df.loc[hold_dist_df['number'] == row['Moves'][move_index - 1], move]
            # convert the distance to an integer
            dist = int(dist)
            # multiply the distance by the difficulty rating of both holds
            diff = dist * df_coordinates.loc[df_coordinates['coordinates'] == row['Moves'][move_index - 1], 'hold_pop_diff'].values[0]
            # if the difficulty rating is larger than the hardest_move column, replace it
            if(diff > df.loc[index, 'hardest_move']):
                df.loc[index, 'hardest_move'] = diff
            # add the distance to the total_move_dist column
            df.loc[index, 'total_move_dist'] += dist
            # if the distance is larger than the largest_move column, replace it
            if(dist > df.loc[index, 'largest_move']):
                df.loc[index, 'largest_move'] = dist

df.head()

,Grade,UserRating,Moves,total_move_dist,largest_move,hardest_move
13935,6C+,3,"[A1, A6, B8, E8, C13, G15, E16, J16, H18, K18]",6281.0,1000.0,851.239281
2883,7A+,0,"[A1, D1, C4, K4, B8, K9, G11, H11, J16, K18]",8939.0,1931.0,1644.995863
3494,8A,3,"[A1, K1, F10, A18, K18]",7585.0,2000.0,1945.689785
18737,6B+,2,"[K1, J6, F8, I8, F12, C13, C16, A18]",4973.0,922.0,827.569204
4105,6C+,3,"[A1, A5, C10, A14, E16, E17, H18]",4141.0,962.0,784.872574


In [42]:
# sort by largest move
df.sort_values(by=['hardest_move'], ascending=False, inplace=True)
df.head(20)

,Grade,UserRating,Moves,total_move_dist,largest_move,hardest_move
17601,7A,3,"[A1, K1, A18, K18]",7584.0,3584.0,3306.586430
6228,8A,3,"[G3, K3, A9, K9, D12, E12, C15, H18]",8543.0,2259.0,2181.003836
16669,8A,0,"[I3, K3, A8, I9, D10, C12, H17, J18]",7377.0,2183.0,2107.627877
2830,7C+,0,"[H1, I1, A5, B5, K6, G7, H7, B9, C9, J11, K11,...",11540.0,2259.0,2030.958929
16246,7C,0,"[J2, K2, A6, D8, K9, D10, D15, D16, K18]",8329.0,2119.0,2028.463517
2336,7B+,3,"[I1, J4, I5, E7, E11, K12, A14, E17, F17, J18]",7620.0,2030.0,2024.808184
16213,7C,0,"[K1, A6, H8, B11, H11, B15, H18]",8835.0,2183.0,2014.028509
15286,7A+,3,"[K1, A6, G6, G8, D11, F13, G16, J18]",6318.0,2183.0,2014.028509
91,6C+,2,"[K1, A6, F6, H6, G10, A11, D13, B15, G15, D16,...",9248.0,2183.0,2014.028509
6872,6B,0,"[A1, K4, H5, F6, C7, B8, A11, E13, I15, J15, J18]",7054.0,2068.0,2011.843238
